<a href="https://colab.research.google.com/github/ryyutku/DSGP/blob/anuk/Future%20Data%20Model/Fill_missing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.api as sm
from scipy.stats import linregress
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit

In [2]:
files = ["Avg_Daily_Sales_Litres_ceypetco.csv","Avg_Daily_Sales_MT_ceypetco.csv","CIEC Data.csv","GDP_historical_data.csv","Import_Data_WITS.csv","Population_colombo.csv","Sales_IOC.csv","Transport_Report_Tables.csv" ]

In [3]:
ciec_df = pd.read_csv(files[2]) # float64

In [4]:
ciec_df['date'] = pd.to_datetime((ciec_df['date']))

In [5]:
ciec_df.columns

Index(['date', 'fuel_consumption', 'petroleum_imports_crudeOil',
       'Taxes_on_Customs_and_Other_Import Duties',
       'Foreign Direct Investments', 'GDP Goods and Services',
       'GDP: Gross National Income', 'Government Debt',
       'New Vehicle Registrations', 'Vehicle Sales', 'Port Stay Duration',
       'Vehicle Sales Asia', 'No.of Vessels Colombo',
       'Imports of Refined Products', 'Colombo port calls',
       'Tax income profits_gains', 'Tax on Export', 'Tax Goods & Services',
       'Tax Road Transport', 'GDP FCE Households', 'Diesel User Price',
       'Petrol User Price', 'Consumption_Oil', 'Sales 90 Octane',
       'Sales 95 Octane', 'Sales Auto Diesel', 'Household_income',
       'Fuel_other_manufacture'],
      dtype='object')

In [6]:
ciec_df_cols = ['date', 'fuel_consumption', 'petroleum_imports_crudeOil',
       'Taxes_on_Customs_and_Other_Import Duties',
       'Foreign Direct Investments', 'GDP Goods and Services',
       'GDP: Gross National Income', 'Government Debt',
       'New Vehicle Registrations', 'Vehicle Sales', 'Port Stay Duration',
       'Vehicle Sales Asia', 'No.of Vessels Colombo',
       'Imports of Refined Products', 'Colombo port calls',
       'Tax income profits_gains', 'Tax on Export', 'Tax Goods & Services',
       'Tax Road Transport', 'GDP FCE Households', 'Diesel User Price',
       'Petrol User Price', 'Consumption_Oil', 'Sales 90 Octane',
       'Sales 95 Octane', 'Sales Auto Diesel', 'Household_income',
       'Fuel_other_manufacture']

In [7]:
ciec_df.dtypes

,0
date,datetime64[ns]
fuel_consumption,float64
petroleum_imports_crudeOil,float64
Taxes_on_Customs_and_Other_Import Duties,float64
Foreign Direct Investments,float64
GDP Goods and Services,float64
GDP: Gross National Income,float64
Government Debt,float64
New Vehicle Registrations,float64
Vehicle Sales,float64


## **Checking the relationships of the columns to see how they must be interpolated**

Checking how the columns are related to other columns, For forecasting the null values

In [8]:
def calculate_category_correlation(df,feature1, feature2):
    # Drop rows where 'feature1' is NaN
    df_valid_feature1 = df.dropna(subset=[feature1])

    # Drop rows where 'feature2' are NaN
    df_valid_feature2 = df.dropna(subset=[feature2], how='all')

    # Find the overlapping date range where both 'feature1' and the 'feature2' exist
    start_date = max(df_valid_feature1['date'].min(), df_valid_feature2['date'].min())
    end_date = min(df_valid_feature1['date'].max(), df_valid_feature2['date'].max())

    # Filter data to include only the overlapping date range
    df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # Compute correlation between 'fuel_consumption' and the aggregated category
    correlation = df_filtered[[feature1, feature2]].corr().iloc[0, 1]

    return correlation

In [9]:
def find_strong_correlations(df, columns, threshold=0.8):
  strong_correlations = {}
  for i in columns:
    for j in columns:
      if i != j:
        correlation = calculate_category_correlation(df, i, j)
        if abs(correlation) >= threshold:
          if i not in strong_correlations:
            strong_correlations[i] = []
          strong_correlations[i].append((j,correlation))


  if strong_correlations:
    print("THE STRONG CORRELATIONS")
    for feature, related_features  in strong_correlations.items():
      for related_feature, corr in related_features:
        print(f"{feature} and {related_feature} have a correlation of {corr:.2f}")
  else:
    print("NO STRONG CORRELATIONS")

  return strong_correlations


In [10]:
strong_correlations = find_strong_correlations(ciec_df,ciec_df_cols)

THE STRONG CORRELATIONS
date and fuel_consumption have a correlation of 0.90
date and GDP Goods and Services have a correlation of 0.91
date and GDP: Gross National Income have a correlation of 0.93
date and Government Debt have a correlation of 0.94
date and Imports of Refined Products have a correlation of 0.88
date and Tax income profits_gains have a correlation of 0.81
date and Tax Goods & Services have a correlation of -0.93
date and GDP FCE Households have a correlation of 0.89
date and Petrol User Price have a correlation of -0.87
date and Consumption_Oil have a correlation of 0.94
date and Sales 90 Octane have a correlation of 0.96
date and Sales 95 Octane have a correlation of 0.83
date and Sales Auto Diesel have a correlation of 0.81
date and Household_income have a correlation of 0.82
fuel_consumption and date have a correlation of 0.90
fuel_consumption and Vehicle Sales have a correlation of 0.83
fuel_consumption and Petrol User Price have a correlation of -0.84
fuel_consum

In [11]:
ciec_df.isnull().mean() *100

,0
date,0.000000
fuel_consumption,28.649535
petroleum_imports_crudeOil,8.747950
Taxes_on_Customs_and_Other_Import Duties,3.007108
Foreign Direct Investments,37.288136
GDP Goods and Services,57.900492
GDP: Gross National Income,57.900492
Government Debt,69.327501
New Vehicle Registrations,0.601422
Vehicle Sales,57.189721


**Null value analysis**\
0-10% - date, consumption_oil

10-30% - fuel_consumption, New Vehicle Registrations (Interpolation will be applied)

30-50% -petroleum_imports_crude_oil, Taxes_on_Customs_and_Other_Import Duties, Imports of Refined Products, Tax income profits_gains, Tax Goods & Services, Diesel User Price, Petrol User Price, Sales 90 Octane, Sales Auto Diesel,

50% + - Foreign Direct Investments, GDP Goods and Services, GDP: Gross National Income, GDP: Gross National Income, Government Debt, Vehicle Sales, Port Stay Duration, Vehicle Sales Asia, No.of Vessels Colombo, Colombo port calls, Tax on Export, Tax Road Transport, GDP FCE Households, Sales 95 Octane, Household_income, Fuel_other_manufacture. (Interplation will be applied)

**Need to cutdown the range from which data is considered, few possible time frames to start with are 1990,1999,2005,2006,2010 **

In [12]:
def get_correlating_list(feature, dict):
  list = []
  values = dict[feature]
  for v1,v2 in values:
    list.append(v1)
  return list

In [13]:
proxy_df = pd.DataFrame()

In [14]:
# Making a copy of ciec_df
df = ciec_df.copy()

## **Scaling The data**

In [15]:
# scaler = StandardScaler()
# m_scaler = MinMaxScaler()

Using z-score scaling (Standard Scaler) for the columns following normal distribution

In [16]:
# z_cols = ['fuel_consumption', 'GDP: Gross National Income', 'GDP Goods and Services',
#           'Foreign Direct Investments', 'Household_income', 'Tax income profits_gains',
#           'Tax on Export', 'Tax Goods & Services', 'Tax Road Transport', 'Government Debt', 'Vehicle Sales']

In [17]:
# df[z_cols] = scaler.fit_transform(ciec_df[z_cols])

Using min-max scaling for price related values

In [18]:
# m_cols = ['Diesel User Price', 'Petrol User Price', 'petroleum_imports_crudeOil','Imports of Refined Products', 'Consumption_Oil']

In [19]:
# df[m_cols] = m_scaler.fit_transform(ciec_df[m_cols])

Adding log transformation for highly skewed data

In [20]:
# l_cols = ['GDP: Gross National Income', 'Foreign Direct Investments', 'Household_income', 'Tax income profits_gains']

In [21]:
# df[l_cols] = np.log1p(df[l_cols])

## **Building Proxy features for the columns using the columns inside the df**

**Linear Regression**

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

def fill_missing_with_regression(df, target_feature, correlated_features):
    df = df.copy()

    correlated_features = [col for col in correlated_features if col != 'date']

    if len(correlated_features) == 0:
        print(f"No valid correlated features for {target_feature}.")
        return df

    missing_mask = df[target_feature].isna()
    valid_data = df.dropna(subset=[target_feature] + correlated_features)

    print(f"Valid data available for {target_feature}: {len(valid_data)} rows")

    if valid_data.empty:
        print(f"Not enough valid data to train regression for {target_feature}.")
        return df

    df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')

    # Apply MinMax scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train = scaler_X.fit_transform(valid_data[correlated_features].astype(float))
    y_train = scaler_y.fit_transform(valid_data[[target_feature]].astype(float))

    model = LinearRegression()
    model.fit(X_train, y_train.ravel())

    X_missing = df.loc[missing_mask, correlated_features].dropna()

    print(f"Missing values to predict for {target_feature}: {len(X_missing)} rows")

    if X_missing.empty:
        print(f"No valid correlated features available to predict missing {target_feature}.")
        return df

    X_missing_scaled = scaler_X.transform(X_missing.astype(float))

    # Predict missing values (scaled)
    predicted_values_scaled = model.predict(X_missing_scaled)

    # Inverse transform to get back real values
    predicted_values = scaler_y.inverse_transform(predicted_values_scaled.reshape(-1, 1)).flatten()

    df.loc[X_missing.index, target_feature] = predicted_values

    return df


In [28]:
strong_correlations.pop('date')

[('fuel_consumption', 0.8980379064063567),
 ('GDP Goods and Services', 0.9123678001007717),
 ('GDP: Gross National Income', 0.9265286159980275),
 ('Government Debt', 0.937401419133558),
 ('Imports of Refined Products', 0.8828305073744521),
 ('Tax income profits_gains', 0.8097488342146448),
 ('Tax Goods & Services', -0.926198376101571),
 ('GDP FCE Households', 0.8907816212623615),
 ('Petrol User Price', -0.86664851830941),
 ('Consumption_Oil', 0.9352428882633094),
 ('Sales 90 Octane', 0.9577905774640867),
 ('Sales 95 Octane', 0.8290028944585407),
 ('Sales Auto Diesel', 0.8133006833969103),
 ('Household_income', 0.8165636911551781)]

In [31]:
for feature in strong_correlations:
  print()
  print("Filling values for...",feature)
  list = get_correlating_list(feature, strong_correlations)
  df = fill_missing_with_regression(df, feature, list[:5])
  print("correlation:",ciec_df['fuel_consumption'].corr(df['fuel_consumption']))
  print("missing values:",df[feature].isnull().mean())
  print()


Filling values for... fuel_consumption
Valid data available for fuel_consumption: 1829 rows
Missing values to predict for fuel_consumption: 0 rows
No valid correlated features available to predict missing fuel_consumption.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... petroleum_imports_crudeOil
Valid data available for petroleum_imports_crudeOil: 1829 rows
Missing values to predict for petroleum_imports_crudeOil: 0 rows
No valid correlated features available to predict missing petroleum_imports_crudeOil.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... GDP Goods and Services
Valid data available for GDP Goods and Services: 1829 rows
Missing values to predict for GDP Goods and Services: 0 rows
No valid correlated features available to predict missing GDP Goods and Services.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... GDP: Gross National Income
Valid data available for GDP: Gross National Income: 18

<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_fe

Missing values to predict for Government Debt: 0 rows
No valid correlated features available to predict missing Government Debt.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... New Vehicle Registrations
Valid data available for New Vehicle Registrations: 1829 rows
Missing values to predict for New Vehicle Registrations: 0 rows
No valid correlated features available to predict missing New Vehicle Registrations.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... Vehicle Sales
Valid data available for Vehicle Sales: 1829 rows
Missing values to predict for Vehicle Sales: 0 rows
No valid correlated features available to predict missing Vehicle Sales.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... Vehicle Sales Asia
Valid data available for Vehicle Sales Asia: 1829 rows
Missing values to predict for Vehicle Sales Asia: 0 rows
No valid correlated features available to predict missing Vehicle Sales Asia.
correlat

<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_fe

Missing values to predict for Diesel User Price: 0 rows
No valid correlated features available to predict missing Diesel User Price.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... Petrol User Price
Valid data available for Petrol User Price: 1829 rows
Missing values to predict for Petrol User Price: 0 rows
No valid correlated features available to predict missing Petrol User Price.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... Consumption_Oil
Valid data available for Consumption_Oil: 1829 rows
Missing values to predict for Consumption_Oil: 0 rows
No valid correlated features available to predict missing Consumption_Oil.
correlation: 0.9999999999999998
missing values: 0.0


Filling values for... Sales 90 Octane
Valid data available for Sales 90 Octane: 1829 rows
Missing values to predict for Sales 90 Octane: 0 rows
No valid correlated features available to predict missing Sales 90 Octane.
correlation: 0.9999999999999998
missing 

<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_features] = df[correlated_features].fillna(method='ffill').fillna(method='bfill')
<ipython-input-22-0392ab3b69c5>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[correlated_fe

#### **Summary of performing regression filling**

In [ ]:
df.isnull().mean() *100

The following columns doesnt have any strong correlations \
'Foreign Direct Investments' \
'Household_income' \
'Colombo port calls' \
'Port Stay Duration' \
'No.of Vessels Colomob' \

## **Building a Demand Proxy Target Variable**

## **Analyzing Data**